<a href="https://colab.research.google.com/github/holehouse-lab/EnsPredict-colab/blob/main/example_notebooks/polymer_property_predictors.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Welcome to IDPEnsemble Predictor

In [1]:
#@title Install Necessary Dependencies
!pip install git+https://git@github.com/idptools/sparrow.git@polymer_predictors --quiet;

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.6/142.6 kB 8.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 476.7/476.7 kB 41.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.5/101.5 kB 13.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.0/58.0 kB 8.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 4.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 4.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 86.6 MB/s eta 0:00:00


In [2]:
#@title File Imports
import sparrow
from sparrow.predictors.batch_predict import batch_predict
from google.colab import data_table
import pandas as pd
from sparrow.sparrow_exceptions import SparrowException
import matplotlib.pyplot as plt
from google.colab import files
import io
from collections import defaultdict
import torch

Error importing GPy.
 If trying to run parrot-optimize, make sure to use `pip install idptools-parrot[optimize]`


In [3]:
#@title Target Sequence
#@markdown <h1>Enter the primary amino acid sequence:</h1>

#@markdown Notes: 

#@markdown - The following predictors implicitly assume this sequence is disordered.

#@markdown - A sequence cannot be provided at the same time as a fasta file.

# define the function that will be called when the form is submitted
def process_form():
    uploaded_data = files.upload()
    uploaded_fasta = list(uploaded_data.keys())[0]
    # Load protein objects
    try:
      protein_objs = sparrow.read_fasta(uploaded_fasta)
    except TypeError as e:
      print(f"Received TypeError: Perhaps you didn't upload a file?")
      raise e

    return protein_objs



sequence = "" #@param {type:"string"}
fasta_file = True #@param {type:"boolean"}


if fasta_file and len(sequence) > 0:
  raise SparrowException(f"Cannot specify both a sequence and a list of sequences from a fasta file simultaneously")

if fasta_file:
    protein_objs = process_form()
    for protein in protein_objs:
        if len(protein) > 750:
            raise SparrowException(f"Sequences longer than 750 residues are not currently supported."\
            "You entered a protein with length {len(protein)} residues")
else:
    if len(sequence) > 750:
        raise SparrowException(f"Sequences longer than 750 residues are not currently supported."\
            "You entered a protein with length {len(sequence)} residues")
    
    protein_objs = sparrow.Protein(sequence)
    

Saving test.fasta to test.fasta


In [6]:
#@title Sequence Property Prediction

#@markdown <h1>Please select the ensemble features you'd like to predict and then press the run cell button: </h1>

#@markdown Predictions can be run on CPU or GPU. Bulk predictions form fasta may be slow on CPU.

radius_of_gyration = True #@param {type:"boolean"}
end_to_end_distance = True #@param {type:"boolean"}
scaling_exponent = True #@param {type:"boolean"}
prefactor = True #@param {type:"boolean"}
asphericity = True #@param {type:"boolean"}

def __gpu_accelerated_fasta_predictions(protein_objs):
    
    dfs = []
    labels = {header : protein.sequence for header,protein in protein_objs.items()}
    lengths = {protein.sequence : int(len(protein)) for header, protein in protein_objs.items()}
    
    dfs.append(pd.DataFrame(list(labels.items()), columns=['fasta_header', 'sequence']))
    dfs.append(pd.DataFrame(list(lengths.items()), columns=['sequence', 'lengths']))
    if radius_of_gyration:
        radius_of_gyrations = batch_predict(protein_objs,batch_size=32,network="rg")
        dfs.append(pd.DataFrame(list(radius_of_gyrations.items()), columns=['sequence', 'radius_of_gyration']))

    if end_to_end_distance:
        end_to_end_distances = batch_predict(protein_objs,batch_size=32,network="re")
        dfs.append(pd.DataFrame(list(end_to_end_distances.items()), columns=['sequence', 'end_to_end_distance']))
                
    if scaling_exponent:
        scaling_exponents = batch_predict(protein_objs,batch_size=32,network="scaling_exponent")
        dfs.append(pd.DataFrame(list(scaling_exponents.items()), columns=['sequence', 'scaling_exponent']))

    if asphericity:
        asphericitys = batch_predict(protein_objs,batch_size=32,network="asphericity") 
        dfs.append(pd.DataFrame(list(asphericitys.items()), columns=['sequence', 'asphericity']))
        
    if prefactor:
        prefactors = batch_predict(protein_objs,batch_size=32,network="prefactor") 
        dfs.append(pd.DataFrame(list(prefactors.items()), columns=['sequence', 'prefactor']))

    merged_df = dfs[0]  # initialize merged_df with the first dataframe

    for df in dfs[1:]:  # iterate over the remaining dataframes
        # merge on 'key' column
        merged_df = pd.merge(merged_df, df, on='sequence')  
        # move 'sequence' column to the end
        merged_df = merged_df[[col for col in merged_df.columns if col != 'sequence'] + ['sequence']] 
    
    return merged_df
    
def __cpu_fasta_predictions(protein_objs):
    dimensions = defaultdict(dict)

    for i, (header, protein) in enumerate(protein_objs.items(), 1):
        dimensions[header].update({"length" : int(len(protein))})
        dimensions[header].update({"sequence" : protein.sequence})

        if radius_of_gyration:
            dimensions[header].update({"radius_of_gyration" : protein.predictor.radius_of_gyration()})
        
        if end_to_end_distance:
            dimensions[header].update({"end_to_end_distance" : protein.predictor.end_to_end_distance()})

        if scaling_exponent:
            dimensions[header].update({"scaling_exponent" : protein.predictor.scaling_exponent()})

        if asphericity:
            dimensions[header].update({"asphericity" : protein.predictor.asphericity()})

        if prefactor:
            dimensions[header].update({"prefactor" : protein.predictor.prefactor()})

    df = pd.DataFrame(dimensions).T
    df = df.rename_axis('fasta_header')
    df = df[[col for col in df.columns if col != 'sequence'] + ['sequence']] 

    return df

def return_fasta_predictions(protein_objs):
    if torch.cuda.is_available():
        df = __gpu_accelerated_fasta_predictions(protein_objs)
    else:
        df = __cpu_fasta_predictions(protein_objs)
        
    return df

def return_individual_predictions(protein):
    dimensions = defaultdict(dict)

    dimensions[f"{protein.sequence}"].update({"length" : int(len(protein))})

    if radius_of_gyration:
        dimensions[f"{protein.sequence}"].update({"radius_of_gyration" : protein.predictor.radius_of_gyration()})
    
    if end_to_end_distance:
        dimensions[f"{protein.sequence}"].update({"end_to_end_distance" : protein.predictor.end_to_end_distance()})
    
    if scaling_exponent:
        dimensions[f"{protein.sequence}"].update({"scaling_exponent" : protein.predictor.scaling_exponent()})
    
    if asphericity:
        dimensions[f"{protein.sequence}"].update({"asphericity" : protein.predictor.asphericity()})

    if prefactor:
        dimensions[f"{protein.sequence}"].update({"prefactor" : 5.5})

    df = pd.DataFrame(dimensions).T
    
    return df

def is_iterable(obj):
    try:
        iter(obj)
        return True
    except TypeError:
        return False


if is_iterable(protein_objs):
    df = return_fasta_predictions(protein_objs)
else:
    df = return_individual_predictions(protein_objs)


display(data_table.DataTable(df, include_index=True, num_rows_per_page=5))


100%|██████████| 1/1 [00:00<00:00, 80.49it/s]


,fasta_header,lengths,radius_of_gyration,end_to_end_distance,scaling_exponent,asphericity,prefactor,sequence
0,tr|A0A452YC24|A0A452YC24_AEGTS Inosine triphos...,205,45.207512,106.939102,0.551764,0.418464,6.336755,MSGAAAAVARVLPKAVTFVTGNAKKLEEVRAILGSSIPFQSLKLDL...
1,tr|A0A453ALH5|A0A453ALH5_AEGTS Lon protease ho...,994,114.309158,287.563934,0.560979,0.410310,6.261775,PKPERQSTPPAHSRLLVQTLESHRPPSESLMLRAAAAAAAAFPARF...
2,tr|A0A453B1F5|A0A453B1F5_AEGTS UMP-CMP kinase ...,214,45.372711,104.782883,0.559540,0.418276,6.591907,WFFVCLMVETLFFMQVVAENMLGDKKVTVVFVLGGPGSGKGTQCSN...


In [9]:
#@title Download the CSV file containing your predictions

#@markdown <h1>Name of the CSV file that you'd like to save</h1>

filename = "test" #@param {type:"string"}

df.to_csv(f"{filename}.csv")

files.download(f'{filename}.csv')



<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [5]:
#@title Plot predicted end-to-end-distribution
bins = 25 #@param {type:"integer"}
if is_iterable(protein_objs):
    raise SparrowException("We don't currently support predicting end-to-end distance distributions from fasta files. Please use the single sequence method in the cell above.")

p_of_Re_R, p_of_Re_P = protein_objs.polymeric.get_predicted_nu_dep_end_to_end_distribution()
plt.hist(p_of_Re_R, weights=p_of_Re_P, linewidth=0.5, edgecolor='k',bins=bins)
plt.xlabel(r"End to End Distance ($\AA$)")
plt.ylabel("Probability")
plt.axvline(protein_objs.predictor.end_to_end_distance(), linestyle="--",color='r',label=r"Mean $R_e$")
plt.legend()
plt.show()

SparrowException: ignored